In [ ]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import pickle
import time
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load my own functions
import keras_to_numpy as ktnp
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw
from kde_training_utilities import kde_load_data
import ddm_data_simulation as ds
import cddm_data_simulation as cds
import boundary_functions as bf

In [ ]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [3]:
import multiprocessing as mp

In [4]:
# Load Model
model_path = '/media/data_cifs/afengler/data/kde/ddm/keras_models/dnnregressor_ddm_06_28_19_00_58_26/model_0' 
ckpt_path = '/media/data_cifs/afengler/data/kde/ddm/keras_models/dnnregressor_ddm_06_28_19_00_58_26/ckpt_0_final'

model = keras.models.load_model(model_path)
model.load_weights(ckpt_path)

# model_path = "/home/tony/repos/temp_models/keras_models/dnnregressor_ddm_06_28_19_00_58_26/model_0"
# ckpt_path = "/home/tony/repos/temp_models/keras_models/dnnregressor_ddm_06_28_19_00_58_26/ckpt_0_final"

# model = keras.models.load_model(model_path)
# model.load_weights(ckpt_path)

# network_path = "/home/tony/repos/temp_models/keras_models/\
# dnnregressoranalytical_ddm_07_25_19_15_50_52/model.h5"

#model = keras.models.load_model(network_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
weights, biases, activations = ktnp.extract_architecture(model)

In [ ]:
weights[0].shape

In [ ]:
ktnp.log_p(np.array([[0.5, 1, .7, 1, 1]]), weights, biases, activations)

In [ ]:
model.predict(np.array([[0.5, 1, .7, 1, 1]]))

In [25]:
def get_params_from_meta_data(file_path  = ''):
    tmp = pickle.load(open(file_path, 'rb'))[2]
    params = []
    for key in tmp.keys():
        if key == 'delta_t':
            break
        if key != 's':
            params.append(key)
    return params   

In [29]:
boundary #= eval('bf.constant')

<function boundary_functions.constant(t=0)>

In [26]:
# Initializations -----
n_runs = 1
n_samples = 2500
feature_file_path = '/media/data_cifs/afengler/data/kde/ddm/train_test_data/test_features.pickle'
mle_out_path = '/media/data_cifs/afengler/data/kde/ddm/mle_runs'

# NOTE PARAMETERS: 
# WEIBULL: [v, a, w, node, shape, scale]
# param_bounds = [(-1, 1), (0.3, 2), (0.3, 0.7), (0.01, 0.01), (0, np.pi / 2.2)]

# my_optim_columns = ['v_sim', 'a_sim', 'w_sim',
#                     'v_mle', 'a_mle', 'w_mle', 'n_samples']

# Get parameter names in correct ordering:
#dat = pickle.load(open(feature_file_path,'rb'))
meta_data_file_path = '/media/data_cifs/afengler/data/kde/ddm/train_test_data/meta_data.pickle'
parameter_names = get_params_from_meta_data(file_path = meta_data_file_path)

param_bounds = [(-1, 1), (0.5, 2), (0.3, 0.7)]


#parameter_names = list(dat.keys())[:-2] # :-1 to get rid of 'rt' and 'choice' here

# Make columns for optimizer result table
p_sim = []
p_mle = []
param_bounds = []

for parameter_name in parameter_names:
    p_sim.append(parameter_name + '_sim')
    p_mle.append(parameter_name + '_mle')
    #param_bounds = param_bounds.append()
    
my_optim_columns = p_sim + p_mle + ['n_samples']

# Initialize the data frame in which to store optimizer results
optim_results = pd.DataFrame(np.zeros((n_runs, len(my_optim_columns))), columns = my_optim_columns)
optim_results.iloc[:, 2 * len(parameter_names)] = n_samples

# define boundary
boundary = bf.constant
boundary_multiplicative = True


# get network architecture
weights, biases, activations = ktnp.extract_architecture(model)

In [7]:
def make_params(param_bounds = []):
    params = np.zeros(len(param_bounds))
    
    for i in range(len(params)):
        params[i] = np.random.uniform(low = param_bounds[i][0], high = param_bounds[i][1])
        
    return params
# ---------------------

In [17]:
# Define the likelihood function
def log_p(params = [0, 1, 0.9], model = [], data = [], ll_min = 1e-29):
    # Make feature array
    feature_array = np.zeros((data[0].shape[0], len(params) + 2))
    
    # Store parameters
    cnt = 0
    for i in range(0, len(params), 1):
        feature_array[:, i] = params[i]
        cnt += 1
    
    # Store rts and choices
    feature_array[:, cnt] = data[0].ravel() # rts
    feature_array[:, cnt + 1] = data[1].ravel() # choices
    
    # Get model predictions
    prediction = np.maximum(model.predict(feature_array), ll_min)
    
    return(- np.sum(np.log(prediction)))  

In [ ]:
param_grid = np.tile(true_params, (data.shape[0], 1))
inp = np.concatenate([param_grid, data], axis=1)

prediction = np_predict(inp, weights, biases, activations)
prediction.sum()

In [12]:
tmp_params = make_params(param_bounds = param_bounds)
boundary_params = {}
ddm_dat_tmp = cds.ddm_flexbound(v = tmp_params[0],
                                    a = tmp_params[1],
                                    w = tmp_params[2],
                                    s = 1,
                                    delta_t = 0.001,
                                    max_t = 20,
                                    n_samples = n_samples,
                                    boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                    boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                    boundary_params = boundary_params)


In [14]:
data_np = np.concatenate([ddm_dat_tmp[0], ddm_dat_tmp[1]], axis = 1)
t = ktnp.log_p(tmp_params, weights, biases, activations, data_np)
#params, weights, biases, activations, data
t

5480.963312381616

In [20]:
# Main loop ----------- TD: Parallelize
for i in range(0, n_runs, 1): 
    
    # Get start time
    start_time = time.time()
    
    # Generate set of parameters
    tmp_params = make_params(param_bounds = param_bounds)
    
    # Store in output file
    optim_results.iloc[i, :len(parameter_names)] = tmp_params
    
    # Print some info on run
    print('Parameters for run ' + str(i) + ': ')
    print(tmp_params)
    
    # Define boundary params
    boundary_params = {}
    
    # Run model simulations
    ddm_dat_tmp = cds.ddm_flexbound(v = tmp_params[0],
                                    a = tmp_params[1],
                                    w = tmp_params[2],
                                    s = 1,
                                    delta_t = 0.001,
                                    max_t = 20,
                                    n_samples = n_samples,
                                    boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                    boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                    boundary_params = boundary_params)
        
    # Print some info on run
    print('Mean rt for current run: ')
    print(np.mean(ddm_dat_tmp[0]))
    
    # Run optimizer standard
    print('running sequential')
    start_time_sequential = time.time()
    out = differential_evolution(log_p, 
                                 bounds = param_bounds, 
                                 args = (model, ddm_dat_tmp), 
                                 popsize = 30,
                                 disp = True)
    elapsed_sequential = time.time() - start_time_sequential
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_sequential)))
    
    # Run optimizer sequential with ktnp
    print('running sequential ktnp')
    start_time_sequential_np = time.time()
    data_np = np.concatenate([ddm_dat_tmp[0], ddm_dat_tmp[1]], axis = 1)
    out_seq_ktnp = differential_evolution(ktnp.log_p, 
                                 bounds = param_bounds, 
                                 args = (weights, biases, activations, data_np), 
                                 popsize = 30,
                                 disp = True)
    elapsed_sequential_np = time.time() - start_time_sequential_np
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_sequential_np)))
    
    # Run optimizer parallel
    print('running parallel')
    start_time_parallel = time.time()
    data_np = np.concatenate([ddm_dat_tmp[0], ddm_dat_tmp[1]], axis = 1)
    out_parallel = differential_evolution(ktnp.log_p, 
                                          bounds = param_bounds,
                                          args = (weights, biases, activations, data_np),
                                          popsize = 30,
                                          disp = True, 
                                          workers = -1)
    elapsed_parallel = time.time() - start_time_parallel
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_parallel)))

    # Print some info
    print('Solution vector of current run: ')
    print(out.x)
    
    print('Solution vector of current run parallel: ')
    print(out_parallel.x)
    
    print('Solution vector of current run seq ktnp')
    print(out_seq_ktnp.x)
    
    print('The run took: ')
    elapsed = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed)))
    
    # Store result in output file
    optim_results.iloc[i, len(parameter_names):(2*len(parameter_names))] = out.x
# -----------------------

# Save optimization results to file
optim_results.to_csv(mle_out_path + '/mle_results_1.csv')

Parameters for run 0: 
[-0.85818865  1.75881296  0.42076234]
Mean rt for current run: 
1.6219672
running sequential
differential_evolution step 1: f(x)= 3691.8
differential_evolution step 2: f(x)= 3669.8
differential_evolution step 3: f(x)= 3642.52
differential_evolution step 4: f(x)= 3630.24
differential_evolution step 5: f(x)= 3629.41
differential_evolution step 6: f(x)= 3618.85
differential_evolution step 7: f(x)= 3618.85
differential_evolution step 8: f(x)= 3607.54
differential_evolution step 9: f(x)= 3607.54
differential_evolution step 10: f(x)= 3605.27
differential_evolution step 11: f(x)= 3602.54
00:01:24
running sequential ktnp
differential_evolution step 1: f(x)= 3677.35
differential_evolution step 2: f(x)= 3654.28
differential_evolution step 3: f(x)= 3645.29
differential_evolution step 4: f(x)= 3645.29
differential_evolution step 5: f(x)= 3632.93
differential_evolution step 6: f(x)= 3605.83
differential_evolution step 7: f(x)= 3605.71
differential_evolution step 8: f(x)= 3605

In [ ]:
# Read in results
optim_results = pd.read_csv(os.getcwd() + '/experiments/ddm_flexbound_kde_mle_fix_v_0_c1_0_w_unbiased_arange_2_3/optim_results.csv')

In [ ]:
data_np.shape

In [ ]:
plt.scatter(optim_results['v_sim'], optim_results['v_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for v
reg = LinearRegression().fit(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))
reg.score(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))

In [ ]:
plt.scatter(optim_results['a_sim'], optim_results['a_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for a
reg = LinearRegression().fit(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))
reg.score(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))

In [ ]:
plt.scatter(optim_results['w_sim'], optim_results['w_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))
reg.score(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))

In [ ]:
plt.scatter(optim_results['c1_sim'], optim_results['c1_mle'])

In [ ]:
# Regression for c1
reg = LinearRegression().fit(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))
reg.score(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))

In [ ]:
plt.scatter(optim_results['c2_sim'], optim_results['c2_mle'], c = optim_results['a_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))
reg.score(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))

In [ ]:
import numpy as np
testing = np.tile(np.array([1,2,3]), (100, 1))

In [ ]:
np.dot(testing.T, testing)

In [ ]:
testing.s

In [ ]:
ddm_dat_tmp = cds.ddm_flexbound(v = tmp_params[0],
                                a = tmp_params[1],
                                w = tmp_params[2],
                                s = 1,
                                delta_t = 0.001,
                                max_t = 20,
                                n_samples = n_samples,
                                boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                boundary_params = boundary_params)